In [ ]:
from joblib import Parallel, delayed
from subprocess import run
import os
from shutil import copyfile,rmtree
import multiprocessing
num_cores = multiprocessing.cpu_count()

try:
    rmtree('/GB100_1/outrigger_wkdir/results')
except:
    pass
os.mkdir('/GB100_1/outrigger_wkdir/results')

def myfun(s3path):
    # parse path for prefix to name outputs
    file_prefix = s3path.split('.')[0].split('/')[-1]
    prefix = '_'.join(file_prefix.split('_')[:2])
    plate = file_prefix.split('_')[1]

    wkdir = f'/GB100_1/outrigger_wkdir/{prefix}'
    output_dir = '/GB100_1/outrigger_wkdir/results'
    results_subdir = f'{output_dir}/{plate}'
    
    # create dir structure
    os.mkdir(wkdir)
    for target_dir in [output_dir, results_subdir]:
        if not os.path.isdir(target_dir):
            os.mkdir(results_subdir)
    
    gtf_file = '/GB100_1/ref/HG38-PLUS/HG38-PLUS/genes/genes.gtf'
    fa_file = '/GB100_1/ref/HG38-PLUS/HG38-PLUS/fasta/genome.fa'
    
    # pull input from s3
    os.chdir('/home/ubuntu/')
    run(['aws', 's3', 'cp', 
         s3path, f'{wkdir}/'])
    
    # run outrigger (approx. 10 min per sample)
    os.chdir(wkdir)
    run(['outrigger', 'index', 
         '--sj-out-tab', f'{file_prefix}.homo.SJ.out.tab',
         '--gtf', gtf_file])
    try:
        os.chdir(wkdir)
        run(['outrigger', 'validate', 
             '--genome', 'hg38',
             '--fasta', fa_file])
    except:
        pass

    # compile results
    for subtype in ['se','mxe']:
        try:
            # /GB100_1/outrigger_wkdir/A10_B000873/outrigger_output/index/se/validated/events.csv
            copyfile(f'{wkdir}/outrigger_output/index/{subtype}/validated/events.csv', 
                     f'{results_subdir}/{prefix}_{subtype}.csv')
        except:
            os.mknod(f'{results_subdir}/{prefix}_{subtype}.csv')
    
    # remove subdir
    rmtree(wkdir)

Parallel(n_jobs=num_cores-1, 
         backend="threading")(map(delayed(myfun), matched_path))
